In [23]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [24]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [25]:
train_data=torchvision.datasets.CIFAR10(root='./data',train=True,transform=transform,download=True)
test_data=torchvision.datasets.CIFAR10(root='./data',train=False,transform=transform,download=True)


train_loader=DataLoader(train_data,batch_size=32,shuffle=True,num_workers=2)
test_loader=DataLoader(test_data,batch_size=32,shuffle=True,num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
image,label=train_data[0]

In [27]:
image

tensor([[[-0.5373, -0.6627, -0.6078,  ...,  0.2392,  0.1922,  0.1608],
         [-0.8745, -1.0000, -0.8588,  ..., -0.0353, -0.0667, -0.0431],
         [-0.8039, -0.8745, -0.6157,  ..., -0.0745, -0.0588, -0.1451],
         ...,
         [ 0.6314,  0.5765,  0.5529,  ...,  0.2549, -0.5608, -0.5843],
         [ 0.4118,  0.3569,  0.4588,  ...,  0.4431, -0.2392, -0.3490],
         [ 0.3882,  0.3176,  0.4039,  ...,  0.6941,  0.1843, -0.0353]],

        [[-0.5137, -0.6392, -0.6235,  ...,  0.0353, -0.0196, -0.0275],
         [-0.8431, -1.0000, -0.9373,  ..., -0.3098, -0.3490, -0.3176],
         [-0.8118, -0.9451, -0.7882,  ..., -0.3412, -0.3412, -0.4275],
         ...,
         [ 0.3333,  0.2000,  0.2627,  ...,  0.0431, -0.7569, -0.7333],
         [ 0.0902, -0.0353,  0.1294,  ...,  0.1608, -0.5137, -0.5843],
         [ 0.1294,  0.0118,  0.1137,  ...,  0.4431, -0.0745, -0.2784]],

        [[-0.5059, -0.6471, -0.6627,  ..., -0.1529, -0.2000, -0.1922],
         [-0.8431, -1.0000, -1.0000,  ..., -0

In [28]:
image.size()

torch.Size([3, 32, 32])

In [29]:
class_names=["plane","car","bird","cat","deer","dog","frog","horse","ship","truck"]

In [30]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3,12,5) # 32- 5 /   1=27+1=28  =====>feature_map=(12,28,28)
        self.pool=nn.MaxPool2d(2,2)# (12,14,14)
        self.conv2=nn.Conv2d(12,24,5) # (24,10,10) -> (24,5,5) -> Flatten (24*5*5)
        self.fc1=nn.Linear(24 * 5 * 5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)  # 1à is the number of classe  in this dataset


    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x 

In [31]:
model=NeuralNet()
loss_function=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

In [32]:
for epoch in range(30):
    print(f'training epch {epoch}')
    running_loss=0
    for i ,data in enumerate(train_loader):
        inputs,labels=data
        optimizer.zero_grad()
        output=model(inputs)
        loss=loss_function(output,labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Loss:{running_loss /len(train_loader):.4f}')


training epch 0
Loss:2.1998
training epch 1
Loss:1.7110
training epch 2
Loss:1.4923
training epch 3
Loss:1.3796
training epch 4
Loss:1.2809
training epch 5
Loss:1.1930
training epch 6
Loss:1.1182
training epch 7
Loss:1.0626
training epch 8
Loss:1.0140
training epch 9
Loss:0.9704
training epch 10
Loss:0.9314
training epch 11
Loss:0.8968
training epch 12
Loss:0.8617
training epch 13
Loss:0.8309
training epch 14
Loss:0.8015
training epch 15
Loss:0.7703
training epch 16
Loss:0.7429
training epch 17
Loss:0.7144
training epch 18
Loss:0.6894
training epch 19
Loss:0.6632
training epch 20
Loss:0.6381
training epch 21
Loss:0.6138
training epch 22
Loss:0.5923
training epch 23
Loss:0.5665
training epch 24
Loss:0.5492
training epch 25
Loss:0.5269
training epch 26
Loss:0.5024
training epch 27
Loss:0.4854
training epch 28
Loss:0.4668
training epch 29
Loss:0.4455


In [33]:
torch.save(model.state_dict(),"model.pth")


In [34]:
net=NeuralNet()
net.load_state_dict(torch.load("model.pth"))

C:\Users\user\AppData\Local\Temp\ipykernel_11532\161103824.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

In [36]:
correct=0
total=0
net.eval()
with torch.no_grad():
    for data in test_loader:
        images,labels=data
        outputs=net(images)
        _,predicted=torch.max(outputs,1)
        total+= labels.size(0)
        correct+= (predicted ==labels).sum().item()

accuracy=100* correct /total
print(f"Accuracy:{accuracy}%")


Accuracy:68.64%


In [38]:
new_transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize((32,32))
])

def load_image(image_path):
    image=Image.open(image_path)
    image=new_transform(image)
    image=image.unsqueeze(0) # add addionnal dimension
    return image

image_paths=["German-Shepherd-dog-Alsatian.webp","PROD-Airplane-flying-above-clouds.webp"]
images=[load_image(img) for img in image_paths]

In [39]:
net.eval()
with torch.no_grad():
    for image in images:
        optimizer.zero_grad()
        output=net(image)
        _,predicted=torch.max(output,1)
        print(f"prediction:{class_names[predicted.item()]}")


prediction:dog
prediction:plane
